https://code-code.tistory.com/16
- 여기서 xgboost 설치

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split
import time
import os

In [2]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:
data_dir = "./private_data/merged_result_data_shift_half_1/data"
file_names = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]

train_df = pd.read_csv('./archive/fashion-mnist_train.csv', sep=',')
test_df = pd.read_csv('./archive/fashion-mnist_test.csv', sep=',')
private_df = pd.read_csv('./Private_data_csv/Private_data/Private_data.csv', sep=',')

train_data = np.array(train_df, dtype = 'float32')
test_data = np.array(test_df, dtype = 'float32')
private_data = np.array(private_df, dtype = 'float32')

X_train = train_data[:, 1:] # 학습데이터
y_train = train_data[:, 0] # 학습데이터의 정답 레이블

X_test = test_data[:, 1:] # 테스트데이터
y_test = test_data[:, 0] # 테스트데이터의 정답 레이블

### 정규화 및 PCA

In [4]:
stdscaler = StandardScaler()
X_train_scale = stdscaler.fit_transform(X_train)
X_test_scale = stdscaler.transform(X_test)
private_scale = stdscaler.transform(private_data)

pca = PCA(n_components=187, random_state=42)
X_train_pca = pca.fit_transform(X_train_scale)
X_test_pca = pca.transform(X_test_scale)
private_pca = pca.transform(private_scale)

### 혼성모델

In [5]:
logi = LogisticRegression(max_iter=200, solver='liblinear')
svm_rbf = svm.SVC(C=13, kernel='rbf', gamma='auto', probability=True)
RF = ensemble.RandomForestClassifier(criterion='entropy', max_depth=70, n_estimators=100)
Gboost = ensemble.GradientBoostingClassifier(n_estimators=100)
XGB = XGBClassifier(use_label_encoder=False, objective='multi:softmax', eval_metric='merror')

In [12]:
vote5_hard = ensemble.VotingClassifier([('logistic',logi),
                                  ('svm_rbf',svm_rbf),
                                  ('RandomForest',RF),
                                  ('Gradient_B',Gboost),
                                  ('XGB', XGB)],
                                  voting='hard',
                                  n_jobs=4, verbose=True)

vote5_hard.fit(X_train_pca, y_train)
vote5_hard_y_pred = vote5_hard.predict(X_test_pca)
vote5_hard_acc = accuracy_score(y_test, vote5_hard_y_pred)
print("voting(최적 SVM, linearSVM, 결정트리) 정확도", vote5_hard_acc)

# 각 test 샘플의 예측 결과를 텍스트 파일에 저장
result_file_path = 'testRsult_Vote5_hard_y.txt'

with open(result_file_path, "w") as f:
    for f_name, prediction in zip(file_names, vote5_hard_y_pred):
        prediction = int(prediction)
        f_name = f_name.replace(".png","")
        f.write(f"{f_name} {prediction}\n")

voting(최적 SVM, linearSVM, 결정트리) 정확도 0.8861


In [13]:
# 각 private 샘플의 예측 결과를 텍스트 파일에 저장
vote5_hard_pivate = vote5_hard.predict(private_pca)
result_file_path = 'testRsult_Vote5_hard_private.txt'

with open(result_file_path, "w") as f:
    for f_name, prediction in zip(file_names, vote5_hard_pivate):
        prediction = int(prediction)
        f_name = f_name.replace(".png","")
        f.write(f"{f_name} {prediction}\n")

In [ ]:
logi = LogisticRegression(max_iter=200, solver='liblinear')
svm_rbf = svm.SVC(C=13, kernel='rbf', gamma='auto', probability=True)
RF = ensemble.RandomForestClassifier(criterion='entropy', max_depth=70, n_estimators=100)
Gboost = ensemble.GradientBoostingClassifier(n_estimators=100)
XGB = XGBClassifier(use_label_encoder=False, objective='multi:softmax', eval_metric='merror')

In [ ]:
vote5_soft = ensemble.VotingClassifier([('logistic',logi),
                                  ('RandomForest',RF),
                                  ('Gradient_B',Gboost),
                                  ('XGB', XGB),
                                  ('svm_rbf',svm_rbf)],
                                  voting='soft',
                                  n_jobs=3, verbose=True)

vote5_soft.fit(X_train_pca, y_train)
vote5_soft_y_pred = vote5_soft.predict(X_test_pca)
vote5_soft_acc = accuracy_score(y_test, vote5_soft_y_pred)
print("voting(최적 SVM, linearSVM, 결정트리) 정확도", vote5_soft_acc)

# 각 test 샘플의 예측 결과를 텍스트 파일에 저장
result_file_path = 'testRsult_Vote5_soft_y.txt'

with open(result_file_path, "w") as f:
    for f_name, prediction in zip(file_names, vote5_soft_y_pred):
        prediction = int(prediction)
        f_name = f_name.replace(".png","")
        f.write(f"{f_name} {prediction}\n")

In [ ]:
# 각 private 샘플의 예측 결과를 텍스트 파일에 저장
vote5_soft_pivate = vote5_soft.predict(private_pca)
result_file_path = 'testRsult_Vote5_soft_private.txt'

with open(result_file_path, "w") as f:
    for f_name, prediction in zip(file_names, vote5_hard_pivate):
        prediction = int(prediction)
        f_name = f_name.replace(".png","")
        f.write(f"{f_name} {prediction}\n")

In [3]:
# 파일 열기 (읽기 모드로 열기)
with open('./testRsult_Vote5_hard_private.txt', 'r') as file:
    lines = file.readlines()  # 파일 내용을 모두 읽어옴

# 각 라인의 숫자를 정수로 변환
converted_lines = []
for line in lines:
    parts = line.split()  # 공백을 기준으로 문자열 분리
    if len(parts) == 2:
        try:
            num = int(float(parts[1]))  # 부동소수점 숫자로 변환 후 정수로 형변환
            converted_lines.append(f"{parts[0]} {num}\n")  # 변환된 값을 리스트에 추가
        except ValueError:
            pass  # 변환 실패 시 넘어감

# 변환된 내용을 파일에 쓰기
with open('./testResult_Vote5_hard_private2.txt', 'w') as file:
    file.writelines(converted_lines)  # 변환된 내용을 파일에 쓰기